In [50]:
import random

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
%matplotlib inline

In [3]:
data_frame = pd.read_csv('../input/chinese/chineseMNIST.csv')

In [4]:
data_frame.head()

,pixel_0,pixel_1,pixel_2,pixel_3,pixel_4,pixel_5,pixel_6,pixel_7,pixel_8,pixel_9,...,pixel_4088,pixel_4089,pixel_4090,pixel_4091,pixel_4092,pixel_4093,pixel_4094,pixel_4095,label,character
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,9,九
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,9,九
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,9,九
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,9,九
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,9,九


In [9]:
chinese_labels = data_frame['character']
chinese_labels.value_counts()

character
九    1000
十    1000
百    1000
千    1000
万    1000
亿    1000
零    1000
一    1000
二    1000
三    1000
四    1000
五    1000
六    1000
七    1000
八    1000
Name: count, dtype: int64

In [150]:
X = data_frame.drop(['label', 'character'], axis=1)
size_ori = int(X.shape[1] ** 0.5)

In [151]:
print(size_ori)

64


In [152]:
X = X.values.reshape(-1,size_ori, size_ori).astype('uint8')

In [162]:
def resize_crop_img(image: np.ndarray,size: int=28):
    # image should be a np 2d-array
    # size: the length of side of resized image
    _, image = cv2.threshold(image,50,255,cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    
    min_x_left_contour = min(contours, key=lambda x: cv2.boundingRect(x)[0])
    max_x_right_contour = max(contours, key=lambda x: cv2.boundingRect(x)[0]+cv2.boundingRect(x)[2])
    min_y_up_contour = min(contours, key=lambda x: cv2.boundingRect(x)[1])
    max_y_down_contour = max(contours, key=lambda x: cv2.boundingRect(x)[1]+cv2.boundingRect(x)[3])
    
    x_left = cv2.boundingRect(min_x_left_contour)[0]
    y_up = cv2.boundingRect(min_y_up_contour)[1]
    x_right = sum(cv2.boundingRect(max_x_right_contour)[::2])
    y_down = sum(cv2.boundingRect(max_y_down_contour)[1::2])
    
    cropped_image = image[y_up:y_down, x_left:x_right]
    
    resized_image = cv2.resize(cropped_image, (size, size))
    
    return resized_image.flatten()

In [166]:
_list = []
for img in X:
    resized_img = resize_crop_img(img)
    _list.append(resized_img)
    
X_res = np.array(_list)

In [168]:
X_resized = pd.DataFrame(X_res)

In [170]:
X_resized.rename(columns=lambda x: f'pixel_{x}', inplace=True)

In [171]:
X_resized

,pixel_0,pixel_1,pixel_2,pixel_3,pixel_4,pixel_5,pixel_6,pixel_7,pixel_8,pixel_9,...,pixel_774,pixel_775,pixel_776,pixel_777,pixel_778,pixel_779,pixel_780,pixel_781,pixel_782,pixel_783
0,0,0,0,0,0,0,0,0,0,0,...,255,255,255,255,58,5,5,5,4,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,14,87,255,255,255,255,225,14,14,14
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,0,0,0,0,0,0,0,0,177,98,...,0,0,0,0,0,0,0,23,254,237
14996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,82,255
14997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,82,255
14998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,82,255


In [172]:
chinese_labels

0        九
1        九
2        九
3        九
4        九
        ..
14995    八
14996    八
14997    八
14998    八
14999    八
Name: character, Length: 15000, dtype: object

In [173]:
data = pd.concat([X_resized, chinese_labels], axis=1)

In [174]:
data.head()

,pixel_0,pixel_1,pixel_2,pixel_3,pixel_4,pixel_5,pixel_6,pixel_7,pixel_8,pixel_9,...,pixel_775,pixel_776,pixel_777,pixel_778,pixel_779,pixel_780,pixel_781,pixel_782,pixel_783,character
0,0,0,0,0,0,0,0,0,0,0,...,255,255,255,58,5,5,5,4,0,九
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,九
2,0,0,0,0,0,0,0,0,0,0,...,87,255,255,255,255,225,14,14,14,九
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,九
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,九


In [175]:
data.to_csv('../input/chinese/train.csv', index=False)